<a href="https://colab.research.google.com/github/aansheeagrwal/Deep-Learning/blob/main/dgx_benchamrking_tf_mirrored_stratergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
tf.config.experimental.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [5]:
tf.__version__

'2.18.0'

In [6]:
tf.test.is_built_with_cuda()


True

In [7]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [8]:
X_train.shape


(50000, 32, 32, 3)

In [9]:
y_train.shape


(50000, 1)

In [10]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]


In [11]:
classes[y_train[3][0]]


'deer'

In [12]:
y_train[:3]


array([[6],
       [9],
       [9]], dtype=uint8)

In [13]:
y_test.shape


(10000, 1)

In [14]:
X_train.shape


(50000, 32, 32, 3)

In [15]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255


In [16]:
from tensorflow import keras

y_train_categorical = keras.utils.to_categorical(y_train, num_classes=10)
y_test_categorical = keras.utils.to_categorical(y_test, num_classes=10)


In [17]:
y_train[0:5]


array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [18]:
y_train_categorical[0:5]


array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [19]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [20]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [22]:
strategy = tf.distribute.MirroredStrategy()


In [23]:
strategy.num_replicas_in_sync


1

In [24]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)


In [ ]:
%%timeit -n1 -r1
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 63s 308ms/step - accuracy: 0.2526 - loss: 2.0855
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 59s 292ms/step - accuracy: 0.3574 - loss: 1.8320
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 91s 339ms/step - accuracy: 0.3843 - loss: 1.7614
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 75s 305ms/step - accuracy: 0.4015 - loss: 1.7153
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 61s 302ms/step - accuracy: 0.4158 - loss: 1.6792
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 66s 323ms/step - accuracy: 0.4259 - loss: 1.6490
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 65s 323ms/step - accuracy: 0.4361 - loss: 1.6228
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 66s 322ms/step - accuracy: 0.4428 - loss: 1.5996
Epoch 9/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 60s 296ms/step - accuracy: 0.4507 - loss: 1.5788
Epoch 10/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 63s 311ms/step - accuracy: 0.4572 - loss: 1.5601
Epoch 11/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 83s 316ms/step - accuracy: 0.4634 - loss: 1.5431
Epoch 12/50
200/200

In [ ]:
%%timeit -n1 -r1
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)